In [180]:
import pandas as pd
import numpy as np


In [181]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv -O data.csv

--2021-09-27 07:26:26--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   6.75M  --.-KB/s    in 0.08s   

2021-09-27 07:26:27 (80.6 MB/s) - ‘data.csv’ saved [7077973/7077973]



In [182]:
df=pd.read_csv('data.csv')
df.columns.T

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [183]:
features=['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']
features

['neighbourhood_group',
 'room_type',
 'latitude',
 'longitude',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [184]:
df_work=df[features]

In [185]:
df_work.dtypes

neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [186]:
df_work.neighbourhood_group.unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [187]:
df_work.room_type.unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

In [188]:
df_work.isnull().sum()

neighbourhood_group                   0
room_type                             0
latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [189]:
df_work=df_work.fillna(0)

In [190]:
df_work.isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [191]:
df_work.neighbourhood_group.unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
      dtype=object)

In [192]:
df_work.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

Question 1

In [193]:
df_work.neighbourhood_group.mode()

0    Manhattan
dtype: object

In [194]:
#splitting the dataset to train, validation and test
from sklearn.model_selection import train_test_split
df_train_full,df_test=train_test_split(df_work,test_size=0.2,random_state=42)
df_train,df_val=train_test_split(df_train_full,test_size=0.25,random_state=42)
print(len(df_train))
print(len(df_test))
print(len(df_val))

29337
9779
9779


In [195]:
y_train=df_train['price'].values
y_test=df_test['price'].values
y_val=df_val['price'].values
print(len(y_train))
print(len(y_test))
print(len(y_val))

29337
9779
9779


In [196]:
del df_train['price']
del df_test['price']
del df_val['price']
df_train.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

Question 2

In [197]:
# Finding the correlation
numerical=list(df_train.dtypes[ df_train.dtypes!='object'].index)
categorical=list(df_train.dtypes[ df_train.dtypes=='object'].index)
print(numerical)
print(categorical)


['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
['neighbourhood_group', 'room_type']


In [198]:
x_corr_matrix=df_train.corr()
print(x_corr_matrix)
df_train.corr()

                                latitude  ...  availability_365
latitude                        1.000000  ...         -0.005891
longitude                       0.080301  ...          0.083666
minimum_nights                  0.027441  ...          0.138901
number_of_reviews              -0.006246  ...          0.174477
reviews_per_month              -0.007159  ...          0.165376
calculated_host_listings_count  0.019375  ...          0.225913
availability_365               -0.005891  ...          1.000000

[7 rows x 7 columns]


,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


HIgh correlation parameters are 
1. reviews_per_month and number_of_reviews
2. calculated_host_listings_count and availability_365

In [199]:
# Making price binary
above_average=(y_train>152).astype(int)
above_average

array([0, 0, 0, ..., 1, 0, 0])

Question 3

In [200]:
# Finding the mutual information 
from sklearn.metrics import mutual_info_score
neighbourhood_group_mutual_info=round(mutual_info_score(df_train.neighbourhood_group,above_average),2)
neighbourhood_group_mutual_info

0.05

In [201]:
room_type_mutual_info=round(mutual_info_score(df_train.room_type,above_average),2)
room_type_mutual_info

0.14

room_type has more mutual information than neighbourhood_group

Question 4

In [202]:
# Creating one hot encoded categorical variables
from sklearn.feature_extraction import DictVectorizer
diction=df_train[categorical].to_dict(orient='records')

In [203]:
dv=DictVectorizer(sparse=False)
dv.fit(diction)
dv.get_feature_names()

['neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [204]:
dv.transform(diction)

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [205]:
diction=df_train.to_dict(orient='records')

In [206]:
dv=DictVectorizer(sparse=False)
dv.fit(diction)
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']

In [207]:
X_train=dv.transform(diction)

In [208]:
X_train.shape

(29337, 15)

In [209]:
diction=df_val.to_dict(orient='records')
X_val=dv.transform(diction)
X_val.shape

(9779, 15)

In [210]:
y_train=above_average
print(y_train.shape)
y_val=(y_val>152).astype(int)
print(y_val.shape)

(29337,)
(9779,)


In [211]:
from sklearn.linear_model import LogisticRegression
# Creating a logistic regression
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [212]:
y_pred=model.predict_proba(X_val)[:,1]
y_pred


array([0.03240075, 0.58137135, 0.39967304, ..., 0.09245741, 0.03385644,
       0.62963935])

In [213]:
y_pred=(y_pred>0.5).astype(int)
print(y_pred)
print(y_val)

[0 1 0 ... 0 0 1]
[0 0 1 ... 0 0 0]


In [214]:
accuracy=round((y_val==y_pred).mean(),2)
accuracy

0.79

In [215]:
def prepare_data(df,features):
  df_work=df[features]
  df_train_full,df_test=train_test_split(df_work,test_size=0.2,random_state=42)
  df_train,df_val=train_test_split(df_train_full,test_size=0.25,random_state=42)
  y_train=df_train['price'].values
  y_test=df_test['price'].values
  y_val=df_val['price'].values
  del df_train['price']
  del df_test['price']
  del df_val['price']
  train_dict=df_train.to_dict(orient='records')
  test_dict=df_test.to_dict(orient='records')
  val_dict=df_val.to_dict(orient='records')
  dv=DictVectorizer(sparse=False)
  dv.fit(train_dict)
  print(dv.get_feature_names())
  X_train=dv.transform(train_dict)
  X_test=dv.transform(test_dict)
  X_val=dv.transform(val_dict)
  y_train=(y_train>152).astype(int)
  y_test=(y_test>152).astype(int)
  y_val=(y_val>152).astype(int)
  return X_train,X_test,X_val,y_train,y_test,y_val

In [216]:
df=pd.read_csv('data.csv')
print(df.head())
df.isnull().sum()
df=df.fillna(0)
df.isnull().sum()

     id  ... availability_365
0  2539  ...              365
1  2595  ...              355
2  3647  ...              365
3  3831  ...              194
4  5022  ...                0

[5 rows x 16 columns]


id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

Question 4

In [217]:
features=['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'availability_365',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'price']
print(features)
X_train,X_test,X_val,y_train,y_test,y_val=prepare_data(df,features)
print(X_train.shape,X_test.shape,X_val.shape,y_train.shape,y_test.shape,y_val.shape)
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train,y_train)
y_pred=model.predict_proba(X_val)[:,1]
y_pred=(y_pred>0.5).astype(int)
accuracy=round((y_val==y_pred).mean(),2)
print(accuracy)

['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'availability_365', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'price']
['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']
(29337, 15) (9779, 15) (9779, 15) (29337,) (9779,) (9779,)
0.79


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Question 5

In [218]:
accuracy_vector=[]
for i in range(0,len(features)-1):
  #features[0:i]
  features_new=features[0:i]+features[i+1:]
  #print(features_new)
  X_train,X_test,X_val,y_train,y_test,y_val=prepare_data(df,features_new)
  #print(X_train.shape,X_test.shape,X_val.shape,y_train.shape,y_test.shape,y_val.shape)
  model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
  model.fit(X_train,y_train)
  y_pred=model.predict_proba(X_val)[:,1]
  y_pred=(y_pred>0.5).astype(int)
  accuracy_vector.append(round((y_val==y_pred).mean(),4))
  #print(accuracy_vector[-1])




['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']
['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


['availability_365', 'calculated_host_listings_count', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


['availability_365', 'calculated_host_listings_count', 'latitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


['availability_365', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [219]:
acc_diff=accuracy_vector-accuracy
res = "\n".join("{} {}".format(x, y) for x, y in zip(features[:-1], acc_diff.round(5)))
print(res)
min=abs(acc_diff).min()
index=np.argwhere(abs(acc_diff)==min)
print(index[0])
features[index[0][0]]

neighbourhood_group -0.0399
room_type -0.0767
latitude -0.0039
longitude -0.004
availability_365 -0.0091
minimum_nights -0.0053
number_of_reviews -0.0033
reviews_per_month -0.0051
calculated_host_listings_count -0.0037
[6]


'number_of_reviews'

Question 6

In [220]:
df_work=df[features]
df_train_full,df_test=train_test_split(df_work,test_size=0.2,random_state=42)
df_train,df_val=train_test_split(df_train_full,test_size=0.25,random_state=42)
y_train=np.log1p(df_train['price'].values)
y_test=np.log1p(df_test['price'].values)
y_val=np.log1p(df_val['price'].values)
del df_train['price']
del df_test['price']
del df_val['price']
train_dict=df_train.to_dict(orient='records')
test_dict=df_test.to_dict(orient='records')
val_dict=df_val.to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dict)
print(dv.get_feature_names())
X_train=dv.transform(train_dict)
X_test=dv.transform(test_dict)
X_val=dv.transform(val_dict)

['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


In [221]:
def my_rmse(y_pred,y_train):
  serr=(y_pred-y_train)**2
  m_serr=serr.mean()
  return np.sqrt(m_serr)


In [222]:
from sklearn.linear_model import Ridge


In [223]:
rmse_val=[]
alpha=[0, 0.01, 0.1, 1, 10]
for val in alpha:
  clf=Ridge(alpha=val)
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_val)
  rmse_val.append(round(my_rmse(y_pred,y_val),3))
print(rmse_val)


[0.497, 0.497, 0.497, 0.497, 0.498]
